In [1]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
path = "amazon_reviews_2.txt"

In [3]:
import pandas as pd
import numpy as np
# !pip install matplotlib
import matplotlib.pyplot as plt
# import seaborn as sns
# color = sns.color_palette()
%matplotlib inline

In [4]:
data = pd.read_csv(path, delimiter="\t")

In [74]:
data.head()
data.columns
data['REVIEW_TEXT'][0]

'When least you think so, this product will save the day. Just keep it around just in case you need it for something.'

In [6]:
data.loc[data["LABEL"] == "__label1__", "LABEL"] = '1'
data.loc[data["LABEL"] == "__label2__", "LABEL"] = '0'

In [7]:
# data.head()

In [8]:
len(data)

9726

In [9]:
cnt_srs=data.groupby(data["LABEL"]).PRODUCT_CATEGORY.value_counts()
# cnt_srs

In [10]:
# plt.figure(figsize=(16,8))
# sns.barplot(cnt_srs.values, cnt_srs.index, alpha=0.8, color=color[1])
# plt.ylabel('Number of Occurrences', fontsize=16)
# plt.xlabel('(Label, Product Category)', fontsize=16)
# plt.title('Label Vs Product Category', fontsize=18)
# plt.xticks(rotation='vertical')
# plt.show()

In [11]:
# cnt_srs = data.groupby(data["LABEL"]).RATING.value_counts()
# cnt_srs
# plt.figure(figsize=(16,8))
# sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[1])
# plt.ylabel('Number of Occurrences', fontsize=16)
# plt.xlabel('(Label, Rating)', fontsize=16)
# plt.title('Label Vs Rating', fontsize=18)
# plt.xticks(rotation='horizontal')
# plt.show()

In [12]:
df1 = data.groupby("LABEL").REVIEW_TEXT
print(df1)
data["TEXT_LENGTH"] = data['REVIEW_TEXT'].apply(len)
data.columns
data['TEXT_LENGTH']

0        116
1        404
2        248
3        212
4        331
        ... 
9721     227
9722     180
9723    1849
9724     187
9725     554
Name: TEXT_LENGTH, Length: 9726, dtype: int64

In [13]:
#average length of labelled data
cnt_srs = data.groupby(["LABEL"]).TEXT_LENGTH.agg(lambda x: sum(x)/len(x))
cnt_srs

LABEL
1    313.780485
Name: TEXT_LENGTH, dtype: float64

In [14]:
#total number of sentences
data['num_sentences'] = data['REVIEW_TEXT'].apply(lambda x: len(str(x).split('.')))
data['num_sentences']

0        3
1        4
2        6
3        5
4        4
        ..
9721     2
9722     3
9723    31
9724     4
9725     6
Name: num_sentences, Length: 9726, dtype: int64

In [15]:
# !pip install textstat
import textstat

In [16]:
#readability score
from textstat.textstat import textstat
#fk score -> readability score

data["FK_Score"] = data["REVIEW_TEXT"].apply(textstat.flesch_kincaid_grade)

In [17]:
#textual processing
# !pip install nltk
import nltk
nltk.download('stopwords')
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to C:\Users\Amar
[nltk_data]     Anikethvarma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
#stopword counts
def stopword_counts(sent):
  summ = 0
  for char in sent.split():
    summ += char in stop_words
  return summ

data['stop_count'] = data['REVIEW_TEXT'].apply(stopword_counts)

In [19]:
data['stop_count']

0        10
1        28
2        25
3        18
4        34
       ... 
9721     14
9722     13
9723    118
9724     13
9725     54
Name: stop_count, Length: 9726, dtype: int64

In [20]:
#caps lock counts
def capital_counts(x):
    summ =0
    for char in x:
        summ+= char in "QWERTYUIOPASDFGHJKLZXCVBNM"
    return summ
data['caps_count'] = data['REVIEW_TEXT'].apply(capital_counts)

In [21]:
data['caps_count']

0        2
1        4
2        6
3        6
4        4
        ..
9721     8
9722     2
9723    97
9724     4
9725     8
Name: caps_count, Length: 9726, dtype: int64

In [22]:
#check if product name matches something written in text
import re
import string
match_list = []

def checkName(title,text):
    matches = []
    for word in title.split():
        #removing punctuation
        word = "".join((char for char in word if char not in string.punctuation))
        #print(word)
        myreg = r'\b'+word+r'\b'
        r = re.compile(myreg, flags=re.I | re.X)
        matches.append(r.findall(text))
    return len(matches)
        

for a,b in zip(data.PRODUCT_TITLE, data.REVIEW_TEXT):
    number_of_matches = checkName(a,b)
    match_list.append(number_of_matches)
    
data["matchesDf"] = match_list

In [23]:
print(data["matchesDf"])
print(data["PRODUCT_TITLE"][0], data["REVIEW_TEXT"][0])

0        7
1       24
2        5
3        6
4       11
        ..
9721    11
9722     7
9723    10
9724     5
9725    10
Name: matchesDf, Length: 9726, dtype: int64
Targus PAUK10U Ultra Mini USB Keypad, Black When least you think so, this product will save the day. Just keep it around just in case you need it for something.


In [24]:
#counting emojis
data["emojis"] = data["REVIEW_TEXT"].apply(lambda x: 1 if ";)" in x.split() or ":)" in x.split() or ":-)" in x.split() else 0)

In [25]:
'''
sentiment classifier
'''

'\nsentiment classifier\n'

In [26]:
#get sentiments
data.loc[data["RATING"] < 3, "RATING"] = 0
data.loc[data["RATING"] > 3, "RATING"] = 1

In [27]:
data.RATING.value_counts()

1    7333
0    1516
3     877
Name: RATING, dtype: int64

In [28]:
data1 = data.loc[data['RATING'] == 1]
print(len(data1))
data2 = data1.sample(frac=0.2, replace=True)
print(len(data2))
data3 = data1 = data.loc[data['RATING'] == 0]

7333
1467


In [29]:
data.columns

Index(['DOC_ID', 'LABEL', 'RATING', 'VERIFIED_PURCHASE', 'PRODUCT_CATEGORY',
       'PRODUCT_ID', 'PRODUCT_TITLE', 'REVIEW_TITLE', 'REVIEW_TEXT',
       'TEXT_LENGTH', 'num_sentences', 'FK_Score', 'stop_count', 'caps_count',
       'matchesDf', 'emojis'],
      dtype='object')

In [30]:
#data

In [31]:
data4 = pd.concat([data2, data3], ignore_index=True)

In [32]:
import csv                             
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from random import shuffle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import numpy as np
# !pip uninstall nltk
# !pip install nltk

import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Amar
[nltk_data]     Anikethvarma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
rawData = data4[["REVIEW_TEXT", "RATING"]]
rawData = [tuple(x) for x in rawData.values]

In [34]:
print(len(rawData))

2983


In [35]:
table = str.maketrans({key: None for key in string.punctuation})
# table

In [36]:
def preProcess(text):
    # Should return a list of tokens
    lemmatizer = WordNetLemmatizer()
    filtered_tokens=[]
    lemmatized_tokens = []
    stop_words = set(stopwords.words('english'))
    text = text.translate(table)
    for w in text.split(" "):
        if w not in stop_words:
            lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))
        filtered_tokens = [' '.join(l) for l in nltk.bigrams(lemmatized_tokens)] + lemmatized_tokens
    return filtered_tokens

In [37]:
featureDict = {} # A global dictionary of features

def toFeatureVector(tokens):
    localDict = {}
    for token in tokens:
        if token not in featureDict:
            featureDict[token] = 1
        else:
            featureDict[token] = +1
   
        if token not in localDict:
            localDict[token] = 1
        else:
            localDict[token] = +1
    
    return localDict

In [38]:
def crossValidate(dataset, folds):
    shuffle(dataset)
    cv_results = []
    foldSize = int(len(dataset)/folds)
    for i in range(0,len(dataset),foldSize):
        classifier = trainClassifier(dataset[:i]+dataset[foldSize+i:])
        y_pred = predictLabels(dataset[i:i+foldSize],classifier)
        a = accuracy_score(list(map(lambda d : d[1], dataset[i:i+foldSize])), y_pred)
        (p,r,f,_) = precision_recall_fscore_support(list(map(lambda d : d[1], dataset[i:i+foldSize])), y_pred, average ='macro')
        #print(a,p,r,f)
        cv_results.append((a,p,r,f))
    cv_results = (np.mean(np.array(cv_results),axis=0))
    return cv_results

In [39]:
def splitData(percentage):
    dataSamples = len(rawData)
    halfOfData = int(len(rawData)//2)
    trainingSamples = int((percentage*dataSamples)/2)
    for (Text, Label) in rawData[:trainingSamples] + rawData[halfOfData:halfOfData+trainingSamples]:
        trainData.append((toFeatureVector(preProcess(Text)),Label))
    for (Text, Label) in rawData[trainingSamples:halfOfData] + rawData[halfOfData+trainingSamples:]:
        testData.append((toFeatureVector(preProcess(Text)),Label))

In [40]:
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC(C=0.01))])
    return SklearnClassifier(pipeline).train(trainData)

In [41]:

def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))

In [42]:
import nltk
nltk.download('wordnet')



[nltk_data] Downloading package wordnet to C:\Users\Amar
[nltk_data]     Anikethvarma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\Amar
[nltk_data]     Anikethvarma\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [66]:
#rawData = []          # the filtered data from the dataset file (should be 21000 samples)
#preprocessedData = [] # the preprocessed reviews (just to see how your preprocessing is doing)
trainData = []        # the training data as a percentage of the total dataset (currently 80%, or 16800 samples)
testData = []    

#print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
 #     "Preparing the dataset...",sep='\n')

# We split the raw dataset into a set of training data and a set of test data (80/20)
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')
splitData(0.8)
# We print the number of training samples and the number of features
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')
print("Mean of cross-validations (Accuracy, Precision, Recall, Fscore): ", crossValidate(trainData, 10))

Now 2983 rawData, 0 trainData, 0 testData
Preparing training and test data...
Now 2983 rawData, 2386 trainData, 597 testData
Training Samples: 
2386
Features: 
44
Training Classifier...


C:\Users\Amar Anikethvarma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training Classifier...


C:\Users\Amar Anikethvarma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training Classifier...


...

C:\Users\Amar Anikethvarma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training Classifier...
Training Classifier...


C:\Users\Amar Anikethvarma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training Classifier...
Training Classifier...
Mean of cross-validations (Accuracy, Precision, Recall, Fscore):  [0.85994398 0.85328215 0.86728051 0.85465633]


C:\Users\Amar Anikethvarma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [51]:
import sys
print(sys.version)
print(nltk.__version__)

3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]
3.7


In [52]:
rawData = data4[["REVIEW_TEXT", "VERIFIED_PURCHASE", "PRODUCT_CATEGORY", "LABEL", "RATING"]]
rawData = [tuple(x) for x in rawData.values]
# print(rawData[0:5])

In [53]:
def splitData(percentage):
    dataSamples = len(rawData)
    halfOfData = int(len(rawData)/2)
    trainingSamples = int((percentage*dataSamples)/2)
    for (Text, verified_Purchase, product_Category, Label, Rating) in rawData[:trainingSamples] + rawData[halfOfData:halfOfData+trainingSamples]:
        trainData.append((toFeatureVector(preProcess(Text), verified_Purchase, product_Category, Label), Rating))
    for (Text, verified_Purchase, product_Category, Label, Rating) in rawData[trainingSamples:halfOfData] + rawData[halfOfData+trainingSamples:]:
        testData.append((toFeatureVector(preProcess(Text), verified_Purchase, product_Category, Label), Rating))

In [54]:
featureDict = {} # A global dictionary of features

def toFeatureVector(Rating, verified_Purchase, product_Category, tokens):
    localDict = {}
    
#Labels

    # featureDict["L"] = 1   
    # localDict["L"] = labels
    featureDict["R"] = 1   
    localDict["R"] = Rating


#Verified_Purchase
  
    featureDict["VP"] = 1
            
    if verified_Purchase == "N":
        localDict["VP"] = 0
    else:
        localDict["VP"] = 1

#Product_Category

    
    if product_Category not in featureDict:
        featureDict[product_Category] = 1
    else:
        featureDict[product_Category] = +1
            
    if product_Category not in localDict:
        localDict[product_Category] = 1
    else:
        localDict[product_Category] = +1
            
            
#Text        

    for token in tokens:
        if token not in featureDict:
            featureDict[token] = 1
        else:
            featureDict[token] = +1
            
        if token not in localDict:
            localDict[token] = 1
        else:
            localDict[token] = +1
    
    return localDict

In [55]:
# trainData = []        # the training data as a percentage of the total dataset (currently 80%, or 16800 samples)
# testData = []    

# #print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
#  #     "Preparing the dataset...",sep='\n')

# # We split the raw dataset into a set of training data and a set of test data (80/20)
# print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
#       "Preparing training and test data...",sep='\n')
# splitData(0.8)
# # We print the number of training samples and the number of features
# print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
#       "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')
# print("Mean of cross-validations (Accuracy, Precision, Recall, Fscore): ", crossValidate(trainData, 20))

In [56]:
classifier = trainClassifier(trainData)
predictions = predictLabels(testData, classifier)
true_labels = list(map(lambda d: d[1], testData))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.8458961474036851
Precision:  0.8446766561514195
Recall:  0.8458961474036851
f1-score:  0.8450925090252708


C:\Users\Amar Anikethvarma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [57]:
print(testData[0])

({'R': ['this card', 'card hilarious', 'hilarious i', 'i send', 'send family', 'family friend', 'friend great', 'great laugh', 'laugh definitely', 'definitely great', 'great way', 'way show', 'show attention', 'attention fun', 'this', 'card', 'hilarious', 'i', 'send', 'family', 'friend', 'great', 'laugh', 'definitely', 'great', 'way', 'show', 'attention', 'fun'], 'VP': 0, 'Office Products': 1, '1': 1}, 1)


In [67]:
print(predictions[0:10], true_labels[0:10])

[1, 1, 1, 1, 0, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [123]:
sample_text = "xxxx :::)))"
sample_rating = '5'
xfTestData = []
xfTestData.append((toFeatureVector('4', 'Y', 'PC', preProcess(sample_text)),sample_rating))

In [124]:
predictionX = predictLabels(xfTestData, classifier)

In [125]:
predictionX

[1]

In [62]:
import pickle

In [63]:
with open('predictor.pickle', 'wb') as f:
  pickle.dump(predictLabels, f)

with open('classifier.pickle', 'wb') as f:
  pickle.dump(classifier, f)

In [64]:
DATA_PATH = "/content"
# predictor_pickle = open(DATA_PATH+'/predictor.pickle','rb')
predictor_pickle = open('predictor.pickle','rb')
best_model2 = pickle.load(predictor_pickle)

# classifier_pickle = open(DATA_PATH+'/classifier.pickle','rb')
classifier_pickle = open('classifier.pickle','rb')

best_model3 = pickle.load(classifier_pickle)

In [65]:
best_model2(xfTestData, best_model3)

[0]